In [1]:
from game_logic.ghostAgents import RandomGhost
from solucion.maxNAgent import MaxNAgent
from game_logic.randomPacman import RandomPacman
from game_logic.PacmanEnvAbs import PacmanEnvAbs
import random
import math
import numpy as np
from game_logic.game_util import process_state
from main import get_default_agents
import timeit 
import pandas as pd

all_layouts = [
        "custom1",
        "custom2",
        "capsuleClassic",
        "contestClassic",
        "mediumClassic",
        "minimaxClassic",
        "openClassic",
        "originalClassic",
        "smallClassic",
        "testClassic",
        "trappedClassic",
        "trickyClassic",
        "mediumGrid",
        "smallGrid"
    ]
    
class TestParams():
    def __init__(self, test_name, layout, pacman_agent, ghost_agent_0, ghost_agent_1):        
        self.test_name = test_name
        self.layout = layout
        self.pacman_agent = pacman_agent
        self.ghost_agent_0 = ghost_agent_0
        self.ghost_agent_1 = ghost_agent_1


In [2]:
def run_test(test_params):  
    t0 = timeit.default_timer()   
    pacman_agent = test_params.pacman_agent
    ghost_agent_0 = test_params.ghost_agent_0
    ghost_agent_1 = test_params.ghost_agent_1
    agents = [pacman_agent, ghost_agent_0, ghost_agent_1]
    agents.extend(get_default_agents(3, 10))    
    done = False
    env = PacmanEnvAbs(agents = agents, view_distance = (2, 2))      
    game_state = env.reset(enable_render= False, layout_name= test_params.layout)
    turn_index = 0
    while (not(done)):    
        action = agents[turn_index].getAction(game_state)
        game_state, rewards, done, info = env.step(action, turn_index)        
        turn_index = (turn_index + 1) % env._get_num_agents()  
    t1 = timeit.default_timer()    
    time = t1-t0  
    assert(game_state.getNumAgents()>=2)
    if game_state.getNumAgents()==2:
        ret = game_state.get_rewards()
        ret.append(np.nan)
    else:
        ret = game_state.get_rewards()[0:3]
    return ret, time

In [3]:
pacman_agent = RandomPacman(index = 0)
ghost_agent_0 = MaxNAgent(index = 1, unroll_type="MCTS", max_unroll_depth=12, number_of_unrolls=6)
ghost_agent_1 = RandomGhost(index = 2)
sample_test = TestParams("PrimerTest", "mediumGrid", pacman_agent, ghost_agent_0, ghost_agent_1)
print(run_test(sample_test))


([-1508, 246.0, nan], 0.2512065973132849)


In [4]:
selected_layouts = [  
        "originalClassic",  
        "mediumGrid",
        "smallGrid"
    ]
pacmans = [("rnd_pcmn",RandomPacman(index = 0)), ("MAXN_pcmn",MaxNAgent(index = 0, unroll_type="MCTS", max_unroll_depth=5, number_of_unrolls=10))]
ghosts0s = [
    ("MAXN_MC_gh0",MaxNAgent(index = 1, unroll_type="MC", max_unroll_depth=5, number_of_unrolls=10)),
    ("MAXN_MCTS_gh0",MaxNAgent(index = 1, unroll_type="MCTS", max_unroll_depth=5, number_of_unrolls=10)),
]
ghosts1s = [
    ("MAXN_MC_gh1",MaxNAgent(index = 2, unroll_type="MC", max_unroll_depth=5, number_of_unrolls=10)),
    ("MAXN_MCTS_gh1",MaxNAgent(index = 2, unroll_type="MCTS", max_unroll_depth=5, number_of_unrolls=10)),
]

In [5]:
all_tests = []
for layout in selected_layouts:
    for pacman_name, pacman_agent in pacmans:
        for ghost0_name, ghost_0 in ghosts0s:
            for ghost1_name, ghost_1 in ghosts1s:
                name = layout+"_"+pacman_name+"_"+ghost0_name+"_"+ghost1_name
                test = TestParams(name, layout, pacman_agent, ghost_agent_0, ghost_agent_1)
                all_tests.append(test)
number_of_runs = 10

In [6]:
results = []
t0 = timeit.default_timer()   
for i in range(len(all_tests)):   
        values = []
        times = []    
        for _ in range(number_of_runs):
            value, time = run_test(all_tests[i])            
            values.append(value)
            times.append(time)
        #compute avg and std and and add to results
        value_avg = np.round(np.mean(np.array(values), axis=0),2)
        time_avg = np.round(np.mean(np.array(times)),2)
        value_std = np.round(np.std(np.array(values), axis=0),2)
        time_std = np.round(np.std(np.array(times)),2)
        result = {'TestName':all_tests[i].test_name,'Time_Avg':time_avg,'Time_Std':time_std, 
                  'P_Value_Avg':value_avg[0], 
                  'P_Value_Std':value_std[0],
                  'GH0_Value_Avg':value_avg[1], 
                  'GH0_Value_Std':value_std[1],
                  'GH1_Value_Avg':value_avg[2], 
                  'GH1_Value_Std':value_std[2]
                  }
        results.append(result)
results = pd.DataFrame(results)
t1 = timeit.default_timer()    
time = t1-t0  
print("All tests time: ", time, "seconds.")    
display(results)

All tests time:  1669.4976891698316 seconds.


,TestName,Time_Avg,Time_Std,P_Value_Avg,P_Value_Std,GH0_Value_Avg,GH0_Value_Std,GH1_Value_Avg,GH1_Value_Std
0,originalClassic_rnd_pcmn_MAXN_MC_gh0_MAXN_MC_gh1,1.69,1.32,-1469.1,40.86,-160.8,390.48,90.5,640.75
1,originalClassic_rnd_pcmn_MAXN_MC_gh0_MAXN_MCTS...,2.44,0.80,-1481.3,24.38,-147.1,404.10,92.5,454.26
2,originalClassic_rnd_pcmn_MAXN_MCTS_gh0_MAXN_MC...,4.43,2.67,-1452.9,61.93,-175.8,335.87,-125.0,484.59
3,originalClassic_rnd_pcmn_MAXN_MCTS_gh0_MAXN_MC...,3.69,1.81,-1467.0,26.54,70.9,505.09,-167.6,302.92
4,originalClassic_MAXN_pcmn_MAXN_MC_gh0_MAXN_MC_gh1,71.14,47.39,-1421.8,209.97,-4530.3,3868.36,-16333.9,12124.85
5,originalClassic_MAXN_pcmn_MAXN_MC_gh0_MAXN_MCT...,22.29,12.60,-1348.2,178.73,-3353.7,2093.19,-11664.8,6740.26
6,originalClassic_MAXN_pcmn_MAXN_MCTS_gh0_MAXN_M...,31.33,27.41,-1503.8,137.73,-5232.3,4762.33,-17298.9,15390.94
7,originalClassic_MAXN_pcmn_MAXN_MCTS_gh0_MAXN_M...,22.86,17.22,-1330.8,154.48,-3615.8,3105.18,-12367.3,9320.65
8,mediumGrid_rnd_pcmn_MAXN_MC_gh0_MAXN_MC_gh1,0.25,0.25,-1515.3,21.28,-96.4,1090.39,NaN,NaN
9,mediumGrid_rnd_pcmn_MAXN_MC_gh0_MAXN_MCTS_gh1,0.13,0.06,-1503.7,4.58,436.1,264.26,NaN,NaN
